# Train and Deploy Your BERT Model with GluonNLP on SageMaker

## Fine-tuning BERT for Sentiment Analysis

In this section, we fine-tune the BERT Base model for sentiment analysis on the IMDB dataset.

## Preparation

First, let's install the necessary dependencies.

In [1]:
# gluonnlp is not available on SageMaker
!pip install mxnet-cu100mkl https://github.com/dmlc/gluon-nlp/tarball/master -U -q
import argparse, time
import mxnet as mx
import gluonnlp as nlp
# a few utitility functions
import utils

mxnet-cu100mkl 1.5.1.post0 has requirement numpy<2.0.0,>1.16.0, but you'll have numpy 1.14.5 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# these hyper-parameters can be used for SageMaker training jobs, or hyper-parameter tuning jobs.
parser = argparse.ArgumentParser(description='BERT sentiment analysis fine-tune hyper-parameters.')
parser.add_argument('--batch_size', type=int, default=32,
                    help='batch size per GPU.')
parser.add_argument('--num_epochs', type=int, default=1, 
                    help='The number of epochs to train')
parser.add_argument('--lr', type=float, default=5e-5,
                    help='Learning rate')

args = parser.parse_args([])
print(args)

Namespace(batch_size=32, lr=5e-05, num_epochs=1)


### Get Pre-trained BERT Model

We can load the pre-trained BERT fairly easily using the model API in GluonNLP, which returns the vocabulary along with the model. We include the pooler layer of the pre-trained model by setting `use_pooler` to `True`.
The list of pre-trained BERT models available in GluonNLP can be found [here](http://gluon-nlp.mxnet.io/model_zoo/bert/index.html).

Now that we have loaded the BERT model, we only need to attach an additional layer for classification.
The `BERTClassifier` class uses a BERT base model to encode sentence representation, followed by a `nn.Dense` layer for classification. We only need to initialize the classification layer. The encoding layers are already initialized with pre-trained weights. 

In [3]:
ctx = utils.try_all_gpus()
bert, vocabulary = nlp.model.get_model('bert_12_768_12', # the 12-layer BERT Base model
                                            dataset_name='book_corpus_wiki_en_uncased',
                                            # use pre-trained weights
                                            pretrained=True, ctx=ctx,
                                            # decoder and classifier are for pre-training only
                                            use_decoder=False, use_classifier=False)

net = nlp.model.BERTClassifier(bert, 2)
# only need initialize the classification layer from scratch
net.classifier.initialize(ctx=ctx)
# compile the model, required for deployment
net.hybridize()
print(net)
loss_fn = mx.gluon.loss.SoftmaxCELoss()

BERTClassifier(
  (bert): BERTModel(
    (encoder): BERTEncoder(
      (dropout_layer): Dropout(p = 0.1, axes=())
      (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
      (transformer_cells): HybridSequential(
        (0): BERTEncoderCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
          (attention_cell): MultiHeadAttentionCell(
            (_base_cell): DotProductAttentionCell(
              (_dropout_layer): Dropout(p = 0.1, axes=())
            )
            (proj_query): Dense(768 -> 768, linear)
            (proj_key): Dense(768 -> 768, linear)
            (proj_value): Dense(768 -> 768, linear)
          )
          (proj): Dense(768 -> 768, linear)
          (ffn): BERTPositionwiseFFN(
            (ffn_1): Dense(768 -> 3072, linear)
            (activation): GELU()
            (ffn_2): Dense(3072 -> 768, linear)
            (dropout_layer): Dropout(p = 0.1, axes=())
            (layer_norm): BERTLayerNorm(eps=1e-12, axis

## Data Preprocessing

To use the pre-trained BERT model, we need to:
- tokenize the inputs into words,
- insert [CLS] at the beginning of a sentence, 
- insert [SEP] at the end of a sentence, and
- generate segment ids

### Data Transformations

We again use the IMDB dataset, but for this time, downloading using the GluonNLP data API. We then use the transform API to transform the raw scores to positive labels and negative labels. 
To process sentences with BERT-style '[CLS]', '[SEP]' tokens, you can use `data.BERTSentenceTransform` API.

In [4]:
train_dataset_raw = nlp.data.IMDB('train')
test_dataset_raw = nlp.data.IMDB('test')
# tokenize texts into words
tokenizer = nlp.data.BERTTokenizer(vocabulary)
# add begin-of-sentence, end-of-sentence tokens and perform vocabulary lookup
transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=128, pair=False)

def transform_fn(data):
    # transform texts to tensors
    text, label = data
    # transform label into position / negative
    label = 1 if label >= 5 else 0
    data, length, segment_type = transform([text])
    return data.astype('float32'), length.astype('float32'), segment_type.astype('float32'), label

In [5]:
train_dataset = train_dataset_raw.transform(transform_fn)
test_dataset = test_dataset_raw.transform(transform_fn)

data, length, _, label = train_dataset[0]
print('original sentence = \n{}'.format(train_dataset_raw[0][0]))
print('\nword indices = \n{}'.format(data.astype('int32')))

original sentence = 
Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!

word indices = 
[    2 22953  2213  4381  2152  2003  1037  9476  4038  1012  2009  2743
  2012  1996  2168  2051  2004  2070  2060  3454  2055  2082  2166  1010
  2107  2

### Let's Train the Model

Now we have all the pieces to put together, and we can finally start fine-tuning the
model with a few epochs.

In [6]:
batch_size = args.batch_size * len(ctx)
train_data, test_data = utils.get_dataloader(batch_size, vocabulary, train_dataset, test_dataset)
tick = time.time()
utils.fit(net, train_data, test_data, args.num_epochs, args.lr, ctx, loss_fn)
tock = time.time()
print('Elapsed time (sec): ', tock-tick)

Batch 0, Train Acc 0.4765625, Train Loss 0.7620674669742584
Batch 25, Train Acc 0.6574519230769231, Train Loss 0.6001719935582235
Batch 50, Train Acc 0.7356004901960784, Train Loss 0.5002635222877941
Batch 75, Train Acc 0.7708675986842105, Train Loss 0.45364915297709796
Batch 100, Train Acc 0.7905321782178217, Train Loss 0.4265145588260476
Batch 125, Train Acc 0.8072916666666666, Train Loss 0.3994708299075091
Batch 150, Train Acc 0.8169495033112583, Train Loss 0.38578553280688277
Batch 175, Train Acc 0.8248845880681818, Train Loss 0.37391214523549104
Epoch 0, Train Acc 0.83128, Train Loss 0.3632569414259372
Test Acc 0.8782526133011799,
Elapsed time (sec):  245.77179074287415


### Inference

In [7]:
utils.predict_sentiment(net, ctx, vocabulary, tokenizer, 'this movie is so great')

'positive'

## Deploy on SageMaker

1. Model parameters
2. Code with data pre-processing and model inference
3. A docker container with dependencies installed
4. Launch a serving end-point with SageMaker SDK

In [8]:
import sagemaker, tarfile, inspect

### 1. Save Model Parameters and Upload to S3

In [9]:
# save parameters, model definition and vocabulary in a zip file
net.export('checkpoint')
with open('vocab.json', 'w') as f:
    f.write(vocabulary.to_json())
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("checkpoint-0000.params") 
    tar.add("checkpoint-symbol.json") 
    tar.add("vocab.json")             
# upload the zip file to s3
session = sagemaker.Session()
uploaded_model = session.upload_data(path='model.tar.gz', key_prefix='model')
s3_model_path = 's3://' + session.default_bucket() + '/model/model.tar.gz'
print("Model was uploaded to", s3_model_path)

Model was uploaded to s3://sagemaker-us-east-1-397262719838/model/model.tar.gz


### 2. the Code for Inference

Two functions: 
1. model_fn() to load model parameters
2. transform_fn() to run model inference given an input

In [10]:
# serve.py contains model_fn() and transform_fn()
!ls serve.py
import serve

serve.py


### 2.1 `model_fn` to Deserialize Checkpoints

In [11]:
print(inspect.getsource(serve.model_fn))

def model_fn(model_dir):
    """
    Load the gluon model. Called once when hosting service starts.
    :param: model_dir The directory where model files are stored.
    :return: a Gluon model, and the vocabulary
    """
    prefix = 'checkpoint'
    net = mx.gluon.nn.SymbolBlock.imports(prefix + '-symbol.json',
                                          ['data0', 'data1', 'data2'],
                                          prefix + '-0000.params')
    net.load_parameters('%s/'%model_dir + prefix + '-0000.params', ctx=mx.cpu())
    vocab_json = open('%s/vocab.json'%model_dir).read()
    vocab = nlp.vocab.BERTVocab.from_json(vocab_json)
    tokenizer = nlp.data.BERTTokenizer(vocabulary)
    return net, vocab



### 2.2 `transform_fn` to Run Model Inference Given an Input

In [12]:
print(inspect.getsource(serve.transform_fn))

def transform_fn(model, data, input_content_type, output_content_type):
    """
    Transform a request using the Gluon model. Called once per request.
    :param model: The Gluon model and the vocab
    :param data: The request payload.
    :param input_content_type: The request content type.
    :param output_content_type: The (desired) response content type.
    :return: response payload and content type.
    """
    # we can use content types to vary input/output handling, but
    # here we just assume json for both
    net, vocabulary, tokenizer = model
    sentence = json.loads(data)
    result = predict_sentiment(net, mx.cpu(), vocabulary, tokenizer, sentence)
    response_body = json.dumps(result)
    return response_body, output_content_type



### 3. Build a Docker Container for Serving

Let's prepare a docker container with all the dependencies required for model inference. Here we build a docker container based on the SageMaker MXNet inference container, and you can find the list of all available inference containers at https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html

Here we use local mode for demonstration purpose. To deploy on actual instances, you need to login into AWS elastic container registry (ECR) service, and push the container to ECR. 

```
docker build -t $YOUR_EDR_DOCKER_TAG . -f Dockerfile
$(aws ecr get-login --no-include-email --region $YOUR_REGION)
docker push $YOUR_EDR_DOCKER_TAG
```

In [13]:
!cat Dockerfile
!docker build -t my-docker:inference . -f Dockerfile -q 

FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.4.1-cpu-py3

RUN pip install mxnet-mkl https://github.com/dmlc/gluon-nlp/tarball/master -U --user

COPY *.py /opt/ml/model/code/sha256:580482ba6a67051181b8cac872f70fcd55723586671fc806fcc50f747bf4b4ff


## Use SageMaker SDK to Deploy the Model

We create a MXNet model which can be deployed later, by specifying the docker image, and entry point for the inference code. If serve.py does not work, use dummy_hosting_module.py for debugging purpose. 

In [14]:
from sagemaker.mxnet.model import MXNetModel
sagemaker_model = MXNetModel(model_data=s3_model_path,    # model parameters
                             image='my-docker:inference', # docker images
                             role=sagemaker.get_execution_role(), 
                             py_version='py3',            # python version
                             framework_version='1.4.1',   # mxnet version
                             entry_point='serve.py',
                             source_dir='.')

We use 'local' mode to test our deployment code, where the inference happens on the current instance.
If you are ready to deploy the model on a new instance, change the `instance_type` argument to values such as `ml.c4.xlarge`

In [15]:
# Here we use 'local' mode for testing, for real instances use c5.2xlarge, p2.xlarge, etc
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='local')

Attaching to tmpkzmd9tww_algo-1-fig7w_1
algo-1-fig7w_1  | 2019-10-09 23:01:56,199 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
algo-1-fig7w_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-fig7w_1  | Current directory: /
algo-1-fig7w_1  | Temp directory: /home/model-server/tmp
algo-1-fig7w_1  | Number of GPUs: 0
algo-1-fig7w_1  | Number of CPUs: 32
algo-1-fig7w_1  | Max heap size: 27305 M
algo-1-fig7w_1  | Python executable: /usr/local/bin/python3.6
algo-1-fig7w_1  | Config file: /etc/sagemaker-mms.properties
algo-1-fig7w_1  | Inference address: http://0.0.0.0:8080
algo-1-fig7w_1  | Management address: http://127.0.0.1:8081
algo-1-fig7w_1  | Model Store: /.sagemaker/mms/models
algo-1-fig7w_1  | Initial Models: ALL
algo-1-fig7w_1  | Log dir: /logs
algo-1-fig7w_1  | Metrics dir: /logs
algo-1-fig7w_1  | Netty threads: 0
algo-1-fig7w_1  | Netty client threads: 0
algo-1-fig7w_1  | Default workers per model: 32
algo-1-fig7w_1  | Blacklist Regex: N/A
algo-1-fig7w_1  | Maximum 

algo-1-fig7w_1  | 2019-10-09 23:01:56,673 [INFO ] pool-1-thread-33 ACCESS_LOG - /172.18.0.1:37396 "GET /ping HTTP/1.1" 200 19
!

In [16]:
output = predictor.predict('The model is deployed. Great!')
print('\nPrediction output: {}\n\n'.format(output))

algo-1-fig7w_1  | 2019-10-09 23:01:58,596 [INFO ] W-9025-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2036
algo-1-fig7w_1  | 2019-10-09 23:01:58,613 [INFO ] W-9009-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2061
algo-1-fig7w_1  | 2019-10-09 23:01:58,614 [INFO ] W-9011-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2054
algo-1-fig7w_1  | 2019-10-09 23:01:58,619 [INFO ] W-9002-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2071
algo-1-fig7w_1  | 2019-10-09 23:01:58,630 [INFO ] W-9003-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2083
algo-1-fig7w_1  | 2019-10-09 23:01:58,635 [INFO ] W-9023-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2083
algo-1-fig7w_1  | 2019-10-09 23:01:58,642 [WARN ] W-9025-model-stderr com.amazonaws.ml.mms.wlm.WorkerLifeCycle - /root/.local/lib/python3.6/site-packages/mxnet/gluon/block.py:1159: UserWarning: Cannot decide type f

### Clean Up

Remove the endpoint after we are done. 

In [17]:
predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


# Resources
- Amazon SageMaker https://aws.amazon.com/sagemaker/
- Amazon SageMaker Python SDK https://sagemaker.readthedocs.io/
- GluonNLP http://gluon-nlp.mxnet.io/
- GluonCV http://gluon-cv.mxnet.io/
- GluonTS https://gluon-ts.mxnet.io/
- Dive into Deep Learning http://d2l.ai/
- MXNet Forum https://discuss.mxnet.io/

## Conclusion

In this tutorial, we showed how to fine-tune sentiment analysis model with pre-trained BERT parameters. In GluonNLP, this can be done with such few, simple steps. All we did was apply a BERT-style data transformation to pre-process the data, automatically download the pre-trained model, and feed the transformed data into the model, deploy the model using SageMaker SDK, all within 50 lines of code!

For more fine-tuning scripts, visit the [BERT model zoo webpage](http://gluon-nlp.mxnet.io/model_zoo/bert/index.html).

## References

[1] Devlin, Jacob, et al. "Bert:
Pre-training of deep
bidirectional transformers for language understanding."
arXiv preprint
arXiv:1810.04805 (2018).

[2] Dolan, William B., and Chris
Brockett.
"Automatically constructing a corpus of sentential paraphrases."
Proceedings of
the Third International Workshop on Paraphrasing (IWP2005). 2005.

[3] Peters,
Matthew E., et al. "Deep contextualized word representations." arXiv
preprint
arXiv:1802.05365 (2018).

[4] Hendrycks, Dan, and Kevin Gimpel. "Gaussian error linear units (gelus)." arXiv preprint arXiv:1606.08415 (2016).

For fine-tuning, we only need to initialize the last classifier layer from scratch. The other layers are already initialized from the pre-trained model weights.